# Line Plots

## 2D line plots

This example plots a sine wave, cosine wave, and ricker wavelet and demonstrates how Graphic Features can be modified by slicing!

In [1]:
import fastplotlib as fpl
from fastplotlib.graphics.selectors import Synchronizer

import numpy as np
from sidecar import Sidecar
from ipywidgets import VBox
from itertools import product

### First generate some data.

In [2]:
# linspace, create 100 evenly spaced x values from -10 to 10
xs = np.linspace(-10, 10, 100)
# sine wave
ys = np.sin(xs)
sine = np.dstack([xs, ys])[0]

# cosine wave
ys = np.cos(xs) + 5
cosine = np.dstack([xs, ys])[0]

# sinc function
a = 0.5
ys = np.sinc(xs) * 3 + 8
sinc = np.dstack([xs, ys])[0]

### We will plot all of it on the same plot. Each line plot will be an individual Graphic, you can have any combination of graphics on a plot.

In [17]:
# Create a plot instance
plot_l = fpl.Plot(size=(600, 300))

# plot sine wave, use a single color
sine_graphic = plot_l.add_line(data=sine, thickness=5, colors="magenta")

# you can also use colormaps for lines!
cosine_graphic = plot_l.add_line(data=cosine, thickness=12, cmap="autumn")

# or a list of colors for each datapoint
colors = ["r"] * 25 + ["purple"] * 25 + ["y"] * 25 + ["b"] * 25
sinc_graphic = plot_l.add_line(data=sinc, thickness=5, colors = colors)

sc = Sidecar(title="lines")

with sc: 
    display(plot_l.show())

RFBOutputContext()

### Graphic features support slicing! :D

In [18]:
# indexing of colors
cosine_graphic.colors[:15] = "magenta"

In [19]:
cosine_graphic.colors[90:] = "red"

In [20]:
cosine_graphic.colors[60] = "w"

In [21]:
# indexing to assign colormaps to entire lines or segments
sinc_graphic.cmap[10:50] = "gray"
sine_graphic.cmap = "seismic"

In [22]:
# more complex indexing, set the blue value directly from an array
cosine_graphic.colors[65:90, 0] = np.linspace(0, 1, 90-65)

### You can capture changes to a graphic feature as events

In [23]:
def callback_func(event_data):
    print(event_data)

# Will print event data when the color changes
cosine_graphic.colors.add_event_handler(callback_func)

In [24]:
# more complex indexing of colors
# from point 15 - 50, set every 3rd point as "cyan"
cosine_graphic.colors[15:50:3] = "cyan"

FeatureEvent @ 0x7f0e24686210
type: colors
pick_info: {'index': range(15, 50, 3), 'collection-index': None, 'world_object': <weakproxy at 0x7f0e2475dcb0 to Line at 0x7f0e2473b3d0>, 'new_data': array([[0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.]], dtype=float32)}



### Graphic data is also indexable

In [25]:
# directly set cosine data from sine data
cosine_graphic.data[10:50:5, :2] = sine[10:50:5]

In [28]:
# change y-values of index 90 on to 7
cosine_graphic.data[90:, 1] = 7

In [30]:
# first index of cosine data = new array 
cosine_graphic.data[0] = np.array([[-10, 0, 0]])

In [31]:
plot_l.close()
sc.close()

### colormaps

In [32]:
plot = fpl.Plot(size=(600, 300))

plot.add_line(sine, thickness=10, name="sine")

sc = Sidecar(title="line plot")

with sc:
    display(plot.show())

RFBOutputContext()

In [33]:
plot["sine"].cmap = "jet"

In [34]:
plot["sine"].cmap.values = sine[:, 1]

In [35]:
plot["sine"].cmap.values = cosine[:, 1]

In [36]:
plot["sine"].cmap = "viridis"

**Qualitative data**

In [37]:
# qualitatively set the cmap values
cmap_values = [0] * 25 + [1] * 5 + [2] * 50 + [3] * 20
plot["sine"].cmap.values = cmap_values

**Qualitative cmap**

In [38]:
# now change the cmap to new cmap with those same cmap values
plot["sine"].cmap = "tab10"

In [39]:
plot.close()
sc.close()

## Line Collections

**Generate some data, 4 x 4 circles**

In [40]:
def make_circle(center, radius: float, n_points: int = 75) -> np.ndarray:
    theta = np.linspace(0, 2 * np.pi, n_points)
    xs = radius * np.sin(theta)
    ys = radius * np.cos(theta)

    return np.column_stack([xs, ys]) + center


spatial_dims = (50, 50)

# this makes 16 circles, so we can create 16 cmap values, so it will use these values to set the
# color of the line based by using the cmap as a LUT with the corresponding cmap_value
circles = list()
for center in product(range(0, spatial_dims[0], 15), range(0, spatial_dims[1], 15)):
    circles.append(make_circle(center, 5, n_points=75))

pos_xy = np.vstack(circles)

### Qualitative cmap

Useful for clustering or classification

In [43]:
plot = fpl.Plot(size=(600, 600))

# things like class labels, cluster labels, etc.
cmap_values = [
    0, 1, 1, 2,
    0, 0, 1, 1,
    2, 2, 3, 3,
    1, 1, 1, 5
]

plot.add_line_collection(
    circles, 
    cmap="tab10", 
    cmap_values=cmap_values, 
    thickness=15
)

sc = Sidecar(title="lines collection")

with sc:
    display(plot.show())

RFBOutputContext()

In [44]:
plot.close()
sc.close()

## neuro data example

In [2]:
# load in movie 
movie = np.load('./fpl-scipy2023-data/neural_data/rcm.npy')

# load in identified nuerons
contours = np.load('./fpl-scipy2023-data/neural_data/contours.npy', allow_pickle=True)

In [3]:
# for the image data and contours
cnmf_iw = fpl.ImageWidget(
    movie, 
    vmin_vmax_sliders=True, 
    grid_plot_kwargs={"size": (600, 500)},
    cmap="gray"
)

# stack the plots and show them in sidecar
sc = Sidecar(title="good/bad comps")

with sc:
    display(cnmf_iw.show())

RFBOutputContext()

/home/clewis7/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


In [4]:
cnmf_iw.vmin_vmax_sliders[0].value = (-5, 15)

### Colors based on classifier output

In [5]:
# indices of accepted_ixs components
accepted_ixs = np.load('./fpl-scipy2023-data/neural_data/good_ixs.npy')

In [6]:
classifier = np.zeros(len(contours), dtype=int)
classifier[accepted_ixs] = 1

classifier

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       1])

In [7]:
# add contours to the plot within the widget
contours_graphic = cnmf_iw.gridplot[0,0].add_line_collection(
    contours, 
    cmap="Set1",
    cmap_values=classifier,
    thickness=3, 
    name="contours"
)

## Red: class 0 - rejected

## Blue: class 1 - accepted

### cmaps for quantitative metrics 

#### CNN classifier output

In [8]:
# load in CNN predictions
cnn_preds = np.load('./fpl-scipy2023-data/neural_data/cnn_preds.npy')

In [9]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

cnn_preds

array([0.99, 0.01, 0.03, 0.02, 0.97, 0.07, 0.16, 0.13, 0.00, 0.00, 0.12,
       0.00, 0.05, 0.45, 0.95, 1.00, 0.33, 0.00, 0.03, 0.00, 0.98, 0.13,
       0.01, 0.03, 0.49, 0.19, 0.98, 0.19, 0.96, 0.51, 0.05, 0.98, 0.01,
       0.08, 0.00, 0.03, 0.03, 0.73, 0.00, 1.00, 0.02, 0.87, 0.83, 0.73,
       0.24, 0.51, 0.78, 0.57, 0.04, 0.25, 0.00, 1.00, 0.99, 0.19, 0.03,
       1.00, 0.01, 0.21, 0.12, 0.99, 0.88, 0.02, 0.04, 0.22, 0.14, 0.58,
       1.00, 0.90, 0.04, 0.12, 0.12, 1.00, 0.36, 0.07, 0.07, 0.22, 0.10,
       0.00, 0.86, 0.12, 0.23, 0.06, 0.68, 0.12, 0.02, 0.02, 0.10, 0.96,
       1.00, 0.13, 0.17, 0.15, 1.00, 0.03, 0.00, 0.87, 0.02, 0.97, 0.94,
       0.06, 0.34, 0.01, 0.74, 0.00, 0.85, 0.08, 0.29, 0.35, 0.79, 1.00,
       0.15, 0.03, 0.45, 0.01, 1.00, 0.96, 0.58, 0.06, 0.00, 0.07, 0.91,
       0.45, 0.99, 0.29, 0.93, 1.00, 1.00, 0.01, 0.03, 0.00, 1.00, 0.00,
       0.98, 0.47, 0.01, 0.03, 0.07, 0.01, 0.85, 0.07, 1.00, 1.00, 0.23,
       0.06, 0.96, 0.49, 0.26, 0.00, 1.00, 0.00, 0.

In [10]:
contours_graphic.cmap = "spring"
contours_graphic.cmap_values = cnn_preds

## low confidence: purple
## high confidence: yellow

#### yet another measure, signal-to-noise measure

In [11]:
# load in SNR comps
snr_comps = np.load('./fpl-scipy2023-data/neural_data/SNR_comps.npy')
np.log10(snr_comps)

array([1.31, 0.99, 0.61, 0.61, 0.79, 0.76, 0.07, 0.21, 0.83, 0.72, 0.51,
       0.07, 0.09, 0.17, 1.22, 0.60, 0.46, 0.62, 0.32, 0.64, 0.61, 0.54,
       0.60, 0.65, 0.30, 0.30, 0.61, 0.32, 1.20, 0.57, 0.46, 1.03, 0.73,
       0.41, 0.20, 0.52, 0.09, -0.15, 1.05, 0.38, 0.42, 1.17, 0.74, 0.42,
       0.32, 0.22, 0.38, -0.07, 0.35, -0.08, 0.71, 0.45, 0.34, 0.03, 0.29,
       0.62, 0.77, 0.51, 0.63, 0.96, 0.63, 0.65, 0.45, 0.32, 0.42, 0.19,
       0.91, 0.78, 0.36, 0.51, 0.32, 1.16, 0.06, 0.46, 0.66, 0.58, 0.34,
       0.47, 0.02, -0.15, 0.14, 0.40, 0.17, 0.16, 0.20, -0.41, 0.11, 0.42,
       1.23, 0.14, 0.23, 0.25, 0.21, 0.50, 0.19, 0.72, 0.48, 0.02, 0.19,
       0.55, 0.11, 0.36, -0.31, 0.57, 0.29, -0.31, 0.24, 1.04, 0.14, 0.56,
       -0.22, 0.39, 0.32, 0.14, 0.88, 0.84, 0.58, 0.42, 0.48, 0.32, 0.85,
       0.22, 0.57, -0.28, 0.61, 1.41, 1.23, 0.64, 1.12, 1.06, 1.09, 0.52,
       0.89, 0.57, 0.53, 0.81, 0.56, 0.36, 0.70, 0.56, 0.97, 1.10, 0.89,
       0.67, 0.79, 0.58, 0.71, 0.57, 0.75,

In [12]:
contours_graphic.cmap_values = np.log10(snr_comps)

## low signal to noise: purple

## high signal to noise: yellow

In [13]:
cnmf_iw.gridplot.close()
sc.close()

## Line Stack - useful for time series data

Generate some sine data

In [14]:
xs = np.linspace(0, 1_000, 2_000)
# sine wave
ys = np.sin(xs) * 20

data = np.vstack([ys] * 500)

In [15]:
data.shape

(500, 2000)

In [16]:
plot = fpl.Plot(size=(600, 500), canvas="jupyter")

# line stack takes all the same arguments as line collection
# and behaves similarly
plot.add_line_stack(data, cmap="jet", separation=15, name="many-lines")

sc = Sidecar(title="line stack")

with sc:
    display(plot.show(maintain_aspect=False))

RFBOutputContext()

## Performant changes

In [17]:
plot["many-lines"].cmap = "viridis"

**more numpy-like indexing :D**

Set the cmap of the first 250 individual lines to `plasma`. 
This sets the cmap _along the line_

In [18]:
plot["many-lines"][:250].cmap = "plasma"

Every 5th datapoint to "white"

In [19]:
plot["many-lines"][:250].colors[::5] = "w"

In [20]:
plot.close()
sc.close()

## Linear Region Selector

Creates a linear region bounded graphic which can be moved along either the x-axis or y-axis.
Allows sub-selecting data from a `Graphic` or from multiple `Graphic` objects.

In [21]:
def interpolate(subdata: np.ndarray, axis: int):
    """1D interpolation to display within the preallocated data array"""
    x = np.arange(0, zoomed_prealloc)
    xp = np.linspace(0, zoomed_prealloc, subdata.shape[0])
    
    # interpolate to preallocated size
    return np.interp(x, xp, fp=subdata[:, axis])  # use the y-values

In [22]:
# data to plot
xs = np.linspace(0, 100, 1_000)
sine = np.sin(xs) * 20
cosine = np.cos(xs) * 20

plot = fpl.GridPlot((5, 1), size=(600, 600))

# preallocated size for zoomed data
zoomed_prealloc = 1_000
zoomed_init = np.column_stack([np.arange(zoomed_prealloc), np.random.rand(zoomed_prealloc)])

# sines and cosines
sines = [sine] * 2
cosines = [cosine] * 2

# make line stack
line_stack = plot[0, 0].add_line_stack(sines + cosines, separation=50)

# make selector
selector = line_stack.add_linear_region_selector()

# populate subplots with preallocated graphics
for i, subplot in enumerate(plot):
    if i == 0:
        # skip the first one
        continue
    # make line graphics for displaying zoomed data
    subplot.add_line(zoomed_init, name="zoomed")


def update_zoomed_subplots(ev):
    """update the zoomed subplots"""
    zoomed_data = selector.get_selected_data()
    
    for i in range(len(zoomed_data)):
        data = interpolate(zoomed_data[i], axis=1)
        plot[i + 1, 0]["zoomed"].data = data
        plot[i + 1, 0].auto_scale()


selector.selection.add_event_handler(update_zoomed_subplots)
plot.show()
sc = Sidecar(title="linear region selector")

with sc: 
    display(plot.show())

RFBOutputContext()

In [23]:
plot.close()
sc.close()

## Heatmap

In [24]:
temporal = np.load('./fpl-scipy2023-data/neural_data/temporal.npy')

In [25]:
# create plot
plot = fpl.GridPlot(shape=(1,2), names=[["heatmap", "timeseries"]])

def normalize(array):
    out = np.zeros(array.shape, dtype=array.dtype)
    for i in range(array.shape[0]):
        a = array[i]
        out[i] = ((a - np.min(a)) / (np.max(a - np.min(a))))
        
    return out

temporal_norm = normalize(temporal)
        
# add temporal traces as heatmap
heatmap = plot[0,0].add_heatmap(temporal_norm)

# add temporal traces as line stack
temporal_stack = plot[0,1].add_line_stack(temporal, colors="magenta")

for subplot in plot:
    subplot.camera.maintain_aspect = False

# heatmap support the same selectors as lines!
heatmap_ls = heatmap.add_linear_selector()
temporal_stack_ls = temporal_stack.add_linear_selector()
temporal_stack_ls.position_z = 10

# the corresponding imaging data and some callbacks to update frames
cnmf_iw = fpl.ImageWidget(movie, vmin_vmax_sliders=True, cmap="gray")

# update the linear selectors with current frame index
def update_linear_selector(change):
    ix = change["new"]
    
    heatmap_ls.selection = ix
    temporal_stack_ls.selection = ix
    
def update_ipywidget(ev):
    ix = ev.pick_info["selected_index"]
    # for line collection sends as list
    if not isinstance(ix, int):
        ix = ix[0]
    cnmf_iw.sliders["t"].value = ix
    
cnmf_iw.sliders["t"].observe(update_linear_selector, "value")
heatmap_ls.selection.add_event_handler(update_ipywidget)
temporal_stack_ls.selection.add_event_handler(update_ipywidget)

    
sc = Sidecar(title="heatmap interactive")

with sc:
    display(VBox([cnmf_iw.show(), plot.show()]))

RFBOutputContext()

RFBOutputContext()

In [26]:
cnmf_iw.vmin_vmax_sliders[0].value = (-5, 15)

In [27]:
plot.close()
sc.close()